In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

C:\Users\arman\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
df = pd.read_csv("cleaned_dataset.csv")
print(df.head())

                                       clean_comment  category  \
0  family mormon have never tried explain them th...         1   
1  buddhism has very much lot compatible with chr...         1   
2  seriously don say thing first all they won get...        -1   
3  what you have learned yours and only yours wha...         0   
4  for your own benefit you may want read living ...         1   

  sentiment_label                                     processed_text  
0        positive  family mormon never tried explain still stare ...  
1        positive  buddhism much lot compatible christianity espe...  
2        negative  seriously say thing first get complex explain ...  
3         neutral  learned want teach different focus goal wrappi...  
4        positive  benefit may want read living buddha living chr...  


In [6]:
nltk.download('vader_lexicon')
df = pd.read_csv("cleaned_dataset.csv")

df['processed_text'] = df['processed_text'].fillna("").astype(str)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\arman\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
sia = SentimentIntensityAnalyzer()

df['scores'] = df['processed_text'].apply(lambda x: sia.polarity_scores(x))
df = pd.concat([df.drop(['scores'], axis=1), df['scores'].apply(pd.Series)], axis=1)

df.head()

,clean_comment,category,sentiment_label,processed_text,neg,neu,pos,compound
0,family mormon have never tried explain them th...,1,positive,family mormon never tried explain still stare ...,0.089,0.459,0.452,0.9349
1,buddhism has very much lot compatible with chr...,1,positive,buddhism much lot compatible christianity espe...,0.110,0.586,0.304,0.9829
2,seriously don say thing first all they won get...,-1,negative,seriously say thing first get complex explain ...,0.064,0.753,0.183,0.7430
3,what you have learned yours and only yours wha...,0,neutral,learned want teach different focus goal wrappi...,0.000,0.909,0.091,0.0772
4,for your own benefit you may want read living ...,1,positive,benefit may want read living buddha living chr...,0.129,0.558,0.313,0.9349


In [16]:
def get_vader_label(compound_score):
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df['vader_pred'] = df['compound'].apply(get_vader_label)

In [12]:
df.to_csv("vader_results.csv", index=False)
print(df.head())

                                       clean_comment  category  \
0  family mormon have never tried explain them th...         1   
1  buddhism has very much lot compatible with chr...         1   
2  seriously don say thing first all they won get...        -1   
3  what you have learned yours and only yours wha...         0   
4  for your own benefit you may want read living ...         1   

  sentiment_label                                     processed_text    neg  \
0        positive  family mormon never tried explain still stare ...  0.089   
1        positive  buddhism much lot compatible christianity espe...  0.110   
2        negative  seriously say thing first get complex explain ...  0.064   
3         neutral  learned want teach different focus goal wrappi...  0.000   
4        positive  benefit may want read living buddha living chr...  0.129   

     neu    pos  compound vader_label vader_sentiment  
0  0.459  0.452    0.9349    positive        positive  
1  0.586  0.304 

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

y_true = df['sentiment_label']
y_pred = df['vader_sentiment']

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

Accuracy: 0.6443
Precision: 0.6622
Recall: 0.6443
F1-score: 0.6484

Classification Report:
              precision    recall  f1-score   support

    negative       0.48      0.64      0.55      8250
     neutral       0.73      0.60      0.66     12778
    positive       0.70      0.68      0.69     15771

    accuracy                           0.64     36799
   macro avg       0.64      0.64      0.63     36799
weighted avg       0.66      0.64      0.65     36799



In [17]:
df['vader_pred'] = df['compound'].apply(get_vader_label)

df_vader_results = df[['processed_text', 'sentiment_label', 'compound', 'vader_pred']]

df_vader_results.to_csv('vader_predictions.csv', index=False)

print("VADER predictions saved to vader_predictions.csv")

VADER predictions saved to vader_predictions.csv
